This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/force.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, Box,
                         Capsule, ContactResults, DiagramBuilder,
                         InverseDynamicsController, JointSliders, LeafSystem,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MultibodyPlant, Rgba, RigidTransform, RollPitchYaw,
                         RotationMatrix, Simulator, Sphere, StartMeshcat,
                         StateInterpolatorWithDiscreteDerivative)

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingXyzJoint, AddShape

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# A simple writing example

A popular example for class projects is to have a robot pick up a pen/pencil/chalk and write or draw something on the board. And that always begs the question "how do we make the actual ink/graphite/chalk lines appear?"

Here is a simple teleop example with a LeafSystem that will help you visualize the drawing in meshcat.  You should be able to write on basically any (one) surface at any orientation.  Go ahead and try making the chalkboard into a sphere. ;-)

You control the chalk, and see what you can draw.  
- X : arrow left/right
- Y : arrow down/up
- Z : key A / Key D

In [ ]:
# Writing System.
class MeshcatWriter(LeafSystem):

    def __init__(self, meshcat, drawing_body_index, canvas_body_index,
                    rgba, line_width=0.01, draw_threshold=0.01):
        LeafSystem.__init__(self)
        self._meshcat = meshcat
        self._drawing_body_index = drawing_body_index
        self._canvas_body_index = canvas_body_index
        self._rgba = rgba
        self._line_width = line_width
        self._draw_threshold = draw_threshold

        self.DeclareAbstractInputPort("contact_results",
                                        AbstractValue.Make(ContactResults()))
        self.DeclarePeriodicDiscreteUpdateEvent(0.01, 0, self.MaybeDraw)

        self._p_WLastDraw_index = self.DeclareDiscreteState(3)
        self._was_in_contact_index = self.DeclareDiscreteState([0])
        self._num_drawn_index = self.DeclareDiscreteState([0])

    def MaybeDraw(self, context, discrete_state):
        results = self.get_input_port().Eval(context)

        # The point on the canvas that most deeply penetrates the chalk.
        p_WDraw = None
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            if (info.bodyA_index() == self._drawing_body_index and
                    info.bodyB_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCb
                break
            elif (info.bodyB_index() == self._drawing_body_index and
                    info.bodyA_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCa
                break

        if p_WDraw is not None:
            p_WLastDraw = context.get_discrete_state(
                self._p_WLastDraw_index).get_value()
            was_in_contact = context.get_discrete_state(
                self._was_in_contact_index)[0]
            num_drawn = context.get_discrete_state(self._num_drawn_index)[0]

            length = np.linalg.norm(p_WDraw - p_WLastDraw)
            if (was_in_contact and length > self._draw_threshold):
                meshcat.SetObject(f"writer/{num_drawn}",
                                    Capsule(self._line_width, length),
                                    self._rgba)
                p_WMidpoint = (p_WDraw + p_WLastDraw)/2
                X_WCapsule = RigidTransform(
                    RotationMatrix.MakeFromOneVector(
                        p_WDraw - p_WLastDraw, 2), p_WMidpoint)
                meshcat.SetTransform(f"writer/{num_drawn}", X_WCapsule)
                discrete_state.set_value(self._num_drawn_index,
                                            [num_drawn + 1])
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)
            elif not was_in_contact:
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)

            discrete_state.set_value(self._was_in_contact_index, [1])
        else:
            discrete_state.set_value(self._was_in_contact_index, [0])


def writing_example():
    builder = DiagramBuilder()

    time_step = 0.001
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
    controller_plant = MultibodyPlant(time_step)
    # Add the chalk to both
    chalk_instance = AddShape(plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
        color=[1, .34, .2, 1.0])
    AddFloatingXyzJoint(plant,
                        plant.GetFrameByName("chalk"),
                        chalk_instance,
                        actuators=True)
    AddShape(controller_plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
        color=[1, .34, .2, 1.0])
    AddFloatingXyzJoint(controller_plant,
                controller_plant.GetFrameByName("chalk"),
                controller_plant.GetModelInstanceByName("chalk"),
                actuators=True)

    # Add a writing surface to the sim plant only
    AddShape(plant, Box(2, 2, .2), "board", color=[0.05, 0.05, 0.05, 1])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("board"),
                     RigidTransform([0, 0, -.1]))

    plant.Finalize()
    controller_plant.Finalize()

    q0 = [0, 0, 0.2]
    plant.SetDefaultPositions(q0)

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat)
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    # InverseDynamicsController
    # TODO(russt): Tune the gains better
    controller = builder.AddSystem(
        InverseDynamicsController(controller_plant,
                                  kp=[20] * 3,
                                  ki=[0] * 3,
                                  kd=[10] * 3,
                                  has_reference_acceleration=False))
    builder.Connect(plant.get_state_output_port(chalk_instance),
                    controller.get_input_port_estimated_state())
    builder.Connect(controller.get_output_port(),
                    plant.get_actuation_input_port())

    # Pose Sliders
    teleop = builder.AddSystem(
        JointSliders(meshcat,
                     plant,
                     q0, [-1, -1, -0.1], [1, 1, 0.2],
                     0.02,
                     decrement_keycodes=['ArrowLeft', 'ArrowDown', 'KeyA'],
                     increment_keycodes=['ArrowRight', 'ArrowUp', 'KeyD']))

    desired_state_from_position = builder.AddSystem(
        StateInterpolatorWithDiscreteDerivative(
            3, time_step, suppress_initial_transient=True))
    builder.Connect(teleop.get_output_port(),
                    desired_state_from_position.get_input_port())
    builder.Connect(desired_state_from_position.get_output_port(),
                    controller.get_input_port_desired_state())

    writer = builder.AddSystem(
        MeshcatWriter(meshcat,
                      plant.GetBodyByName("chalk").index(),
                      plant.GetBodyByName("board").index(),
                      Rgba(1, .34, .2, 1.0), line_width=0.005))
    builder.Connect(plant.get_contact_results_output_port(),
                    writer.get_input_port())

    # Simulate.
    diagram = builder.Build()

    simulator = Simulator(diagram)

    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(0.1)

writing_example()

In [ ]:
51/255